In [ ]:
# default_exp core

# espiownage core

> Core espiownage parts

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import cv2
import numpy as np
from pathlib import Path
import os

In [ ]:
#export
def mkdir_if_needed(path):
    try:                # go ahead and try to make the the directory
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:  # ignore error if dir already exists
            raise

In [ ]:
#export 
def meta_to_img_path(meta_file, # filename of .csv file with annotations
    img_bank='images/',         # alternate location of image files, if not in same dir as meta file
    ):
    '''Suggest the image file that corresponds with an annotation CSV file'''
    meta_file, img_dir = Path(meta_file), Path(os.path.expanduser(img_bank))
    img_file = meta_file.with_suffix('.png')  # check same directory as meta first
    if os.path.exists(img_file): return img_file
    return img_dir / Path(img_file.name)  # return from image storage directory

In [ ]:
meta_to_img_path('annotations/06241902_proc_00023.csv')

PosixPath('images/06241902_proc_00023.png')

In [ ]:
#export 
def meta_to_mask_path(
    meta_file, # filename of .csv file with annotations
    mask_dir='masks/',  # output directory; assumed to exist
    ):
    "provides name for segmentation mask file"
    csv_path = Path(meta_file)
    return Path(mask_dir + csv_path.stem+'_P.png')  # _P because that's what CAMVID dataset does  

In [ ]:
meta_to_mask_path('annotations/06241902_proc_00023.csv')

PosixPath('masks/06241902_proc_00023_P.png')

In [ ]:
#export 
def fix_abangle(a,b,angle):
    "Makes sure semimajor axis > semiminor axis, and angles are consistent"
    if b > a:
        a, b, angle = b, a, angle+90
    if angle < 0: angle += 180
    elif angle >= 180: angle -= 180
    return a, b, angle

In [ ]:
fix_abangle(5,10,-20)

(10, 5, 70)

In [ ]:
#export 
def draw_ellipse(
    img,     # a cv2 image, not a PIL image?
    center, axes, angle,
    startAngle=0, endAngle=360, color=(0) ,
    thickness=2, lineType=cv2.LINE_AA, shift=10, filled=False):
    """"Draws an ellipse into image.
    startAngle & endAngle should are arc-angles. They should stay at 0 & 360 for full ellipses.
    """
    center = [int(round(x* 2**shift)) for x in center]  
    axes = [int(round(x* 2**shift)) for x in axes]
    thickness = -1 if filled else thickness
    ellipse = cv2.ellipse(
        img, center, axes, -angle,   # -angle because the web interface is "upside down"
        startAngle, endAngle, color,
        thickness, lineType, shift)
    return ellipse

In [ ]:
height, width = 512,384
img = np.zeros((width, height), dtype=np.uint8)  # numpy w/h is "backwards" to images
img = draw_ellipse(img, (100,100), (80,30), 45, color=100, filled=True)